# EMNIST

In [1]:
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *

/home/nislah/.vmgr_repo/dev-2021-02-py38/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)

cuda


In [3]:
# Create the working directory from timestamp and model name
model_name = 'cvae_emnist' # invertible network
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
workingDir = f'tb_logs/{model_name}_{timestamp}'
os.mkdir(workingDir)

In [4]:
lr = 1e-3
batch_size = 64
latent_dim = 4
TRAIN_CVAE = False

In [5]:
transform = transforms.Compose([transforms.ToTensor()])#,  transforms.Resize((32,32))]) # Dataset transform #  lambda x: (x>0.5).float(),
dataset = datasets.EMNIST(root='/share/datasets', download=False, split='letters', train=True, transform=transform)
train_set, val_set = torch.utils.data.random_split(dataset, [100000, 24800])
test_set = datasets.EMNIST(root='/share/datasets', download=False, split='letters', train=False, transform=transform)

# Dataloaders
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=12, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=12, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=12, shuffle=False)

In [6]:
# !tensorboard --logdir=./tb_logs
tb_logger = TensorBoardLogger('tb_logs', name=model_name)
enc_layers = [28*28, 512]
dec_layers = [512, 28*28]
if TRAIN_CVAE:
    model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
    trainer = pl.Trainer(gpus=1,logger=tb_logger,max_epochs=100,auto_lr_find=True)
    trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)